# Job Market Assistant

TODO: description


In [9]:
from IPython.display import Markdown
import time
from core.models import ChatGPT, Gemini, Llama
from core.job_market_assistant import JobMarketAssistant
import pandas as pd

## Framework

Our class-based structure allows to encapsulate reused code and perform experiments easily by adding new models and altering the prompts.


All the models have to implement _complete_ method. With parameter _debug_ set to _True_ all the querying history is visible.


In [4]:
debug_chat_gpt = ChatGPT(True)
completion = debug_chat_gpt.complete("Hello Chat!")

>>> ChatGPT: Hello Chat!
ChatGPT >>>: Hello there! How are you doing today?


The system uses 5 different prompts each serving unique purpose and having a distinct set of available parameters:

| Method             | Used prompt           | Purpose                                                                   | Parameters                             |
| ------------------ | --------------------- | ------------------------------------------------------------------------- | -------------------------------------- |
| get_cover_letter   | cover_letter_prompt   | Generate a cover letter.                                                  | job_desc, cv                           |
| get_missing_skills | dream_job_prompt      | Generate a title of the dream job from the CV if not provided explicitely | cv                                     |
| get_missing_skills | key_points_prompt     | Summarize key points for a position.                                      | job_title, company, job_desc, location |
| get_missing_skills | missing_skills_prompt | Identify skills gaps.                                                     | cv, key_points                         |
| get_missing_skills | postprocess_prompt    | Further refine the result of Missing Skills prompt                        | missing_skills                         |


In [2]:
chat_gpt = ChatGPT()

job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    dream_job_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
    postprocess_prompt="Deduplicate the following list of skills and limit it to 10 items: {missing_skills}",
)

## Data sources

Provide the scrapped job postings directly or use one of the helper methods to scrap them from LinkedIn.


In [3]:
file_job_postings = pd.read_feather("data/job_postings.feather")
file_job_postings

,job_title,company,job_desc,location
39,Applied Data Scientist,dunnhumby,dunnhumby is the global leader in Customer Dat...,"London, England, United Kingdom"
47,Data Scientist,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom"
267,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom"
147,"Software Development Engineer, Caching",myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland"
265,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom"
...,...,...,...,...
183,Sales Business Analyst,Aer Lingus,Your role\n\nAre you ready to make a significa...,"Dublin, County Dublin, Ireland"
20,"Software Development Engineer, AWS Security Se...",myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland"
81,Application & Service Engineer,Altus Group Ltd,I am pleased to announce that we are looking f...,"Cork, County Cork, Ireland"
327,Graduate Data Scientist,Worldpanel by Kantar,"We’re the world’s leading data, insights, and ...","Greater London, England, United Kingdom"


In [7]:
job_posting_urls = (
    "https://www.linkedin.com/jobs/view/3937215366/?alternateChannel=search&refId=pyO8PX6NFy5hyGNWQ5uRxQ%3D%3D&trackingId=aNQSGBRWWobujlSYEpbvaQ%3D%3D",
    "https://www.linkedin.com/jobs/view/3931218091/?alternateChannel=search&refId=%2BwGMIZDUEBj8w%2BcBBmKX0g%3D%3D&trackingId=MKGRxpSmYvT7yYfc0vnezg%3D%3D",
    "https://www.linkedin.com/jobs/view/3939354205/?alternateChannel=search&refId=jzm6ENFRWaolwGXJgicMTw%3D%3D&trackingId=Ryft9wMmyemu5LZP3YMVkQ%3D%3D&trk=d_flagship3_search_srp_jobs",
    "https://www.linkedin.com/jobs/view/3933974972/?alternateChannel=search&refId=jzm6ENFRWaolwGXJgicMTw%3D%3D&trackingId=Nf75Lj8H45f%2F3Wn0YZWGow%3D%3D&trk=d_flagship3_search_srp_jobs",
    "https://www.linkedin.com/jobs/view/3706100131/?eBP=CwEAAAGP2nLSS1cVH9CpQXLJCQYoBELJe_DMnb_FF0wnPFmYKZ8RevBvThpD_ekhE_mtBIX8fy5hK5ayxs7uQI8M551BR4OaZ0Vk6wIa_yi-E6P_orjzE4PKlU6nMCHwWnQoskuS4a0jCI8057qk3wYLKiI8pZ_mITVGGc_tOrpA5WVKMj_Zeoa44P2nxT1vJ1KUFmyeIiSBOAnIdhZu4fTCqmELBiPkEAD5eKftXGAuDr3pbYOWdt3TZ9lCMf-xY0rQGFaYih57c03_fADVuhjZygq2VkyoJpNIwGj_5INh7nT_fjMLCs_1POakqPL3o3-H7t2RAdFyjMtjYZ0D3_2500LfftGIcnS0Rrjl9Xss1xgu1HcRoGWs-Yx0hkoolePWe-KCNgbczGhVVMQkW3TEwVYwNmq-5ucGIuEvbNhqYo9H2th1MA7NLumAVxtEIWH6IK8TLe7JiAhf8VBblyQm2RgHaZTO&refId=ejemPX0qrd8qnaCrjBlNWw%3D%3D&trackingId=nRFdwwj3o5%2B%2B7KFMCCFo1w%3D%3D&trk=flagship3_search_srp_jobs",
    "https://www.linkedin.com/jobs/view/3926449844/?eBP=CwEAAAGP2nOgU8_sfnyPI_cLUth9VbgP3dIs3RClQtIwUPjlvFsnHuQAiIaEqWo-DXN0PMr8OSoQIj3obkACHeuwBugF67GnD_Z_IL8WkR2I8yAYJM3taMePkxBmPwI2zsd6facjFtS8XDaha22A0tXYebVPQscG8pvSd5vZSzyVY3r4GN9iBpOrZsgnD6Ql3f-o0NpW4H_Ea-GRZoVD5abwhA_FkiQjPW1_JL_b-MXYWhVlPXImXCYdh2s86zuIfyhmxYCto6nfZoLqgBGfNGa_g5NXMP7cF4tj57CaZxE2Bm22qe_vCdTfkUoiwWNVUa3rSEEtzB49dimLquOca5B2-Sa8IsYSJFTcpWw4GOInhexoSnI-Uawuu1AH_BBLQ-NqGXSAmpoIpbQKtCSID8PnSgUTJx_k4htjYF7FrG7ciSLAhVULZSLkCQ-UNYj9ubHhOzjuGpE1zvNnNKflzOLkmpXUiMg5USw&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=D9AGbek05Cnb7pxMYxiqpg%3D%3D&trk=flagship3_search_srp_jobs",
    "https://www.linkedin.com/jobs/view/3904363506/?eBP=CwEAAAGP2nOgU4zBnhBLcSx9mV0FYGVGgNyQ1sxXETXxVxPVFk53FLqXR9gH9f4U2Nqac6fmAWSBzAAnCbbBO-YY8F0YdhOwtuIQY01HY_W4sWI49I8PJP6pBSKCHPSDOj4lmCGZlDuy8LguqiBqKKAvNjUMPRw6yWT83z-Xo4YwNA62ju020hTZ-YFlK-WATuOZGhVoK0iwbySo89vy-cf2j5ZP7vpLrArpWUtzjNrGCItD_qOoCubYiF_93A_axtk8RkbmwmTIxL7AOl5Ql_veoGeNXdfdiw5wDLiGZBwp1lxvOIFiC_X7655cFQqm5jhB8p7jg8bRSYKSwbawikJntoTMDxs6TpvqjYHnbsT9xKaaPOpgDes3A4U3JONRtNa5S9y_-C3v9QWMC1DwwuPAqcETf7Llf47r4Npo-dxM17YgHj3FQicUzkI8wYSKpmzU0uK44dASHScYXwN5TWVX3pZxqEprZNY&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=vI35XF233bfRnGy6BAE%2FSQ%3D%3D&trk=flagship3_search_srp_jobs",
    "https://www.linkedin.com/jobs/view/3910213820/?eBP=CwEAAAGP2nOgU4o3HMYGtDJlo8CcLUUFwd-jc9D9_hbJ5mLrtYZcutRbb5mRXvtMJ1Ig92XZ0FOxY5SkMcrAfg3VK1tY1JObDFpgTe_y9Y7iNKbX94lMzuYnYBJn21Eeymf77lmdICyU6QH-yPQbdcf33E72AC_xt7EBxnpOv9M0sGXqJc11puvSVQGbXpW5W5Sbbsv6QTzs9FxlovRKu6pEPDCCVzjrHrC9d_gOylePjmUOnabDFkppFa3gbB8zh_il2KSTYB_lhCtl1Kxi-5y0NcsECFB5OR5KFUr0npWo7btVAewY5QBeyUFcbcX2O9pXjVdImXT40xQBaQbIUbZXEZeSEaFroWyIc-kB8FzaKMlLmIK5ezXmgfdGx_f10lWV_9xSFVlvaWI1MeG5adcsO0fTM2fY0ucITXYFS2qbispOnegSv3cS_p4WqGTNkLxg_Nxjr8ppKq8HMMZud8dppz5Izp2lOyg&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=1OHZffvErAg6ayGGSMaGSQ%3D%3D&trk=flagship3_search_srp_jobs",
    "https://www.linkedin.com/jobs/view/3934612685/?eBP=CwEAAAGP2nOgUxurolZNvR-9t8O15gg8Hg73vyxdiM7INtIvQbruj1Lad7exeumIa4mrXe73RR3QJ3t-RkVakkFZqmPBu1QNpw4oyXWLO0A0d7RG_uAQJ0YJCdq34tT2c50_4HPgem5RvZ0Q_KcjsPUZuVDkiy0i5xxjusB-K9fzpJ1kHeoLJVsY4-Hdgjrd8hdvpQ6XwtPErHWtPsHmaBPgMvJfGw22Nu_SW6Or4WeOmLmaBroXMpeJGuVz7UqETkN7ro6h77j_Rsi25h8wYeL7S-NY6zlt5HCltq-cIXuFW_kpQ_FXPqHvnKuXkKpKchCBqgCSa3AUTCEaNF60I50pHS9xNHnhro0RaMODYkEAB1YhKBDLW_JY_qWIORfrs48oEWG5HCt_yCwEQCzO6ydupfJkjLOgCjEpOc3wFSMK6knh7yySP4zbSOArdmbPkMsaTTvgFHCG3loj9FRN49Dbrl5RFd1C3rg&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=xDb049gA1gGIWmWSmZ6jfg%3D%3D&trk=flagship3_search_srp_jobs",
    "https://www.linkedin.com/jobs/view/3903916300/?eBP=CwEAAAGP2nWWOL08uhH7U-M5C0J0CphPf6gFN4SMMBAXxYf-YdOwMoRtXj0UBwSmzeTRcrgSE8fpf7yZ0xy0_rHcDoJxAtbx85i8FJ3WTzcEZNK18GYnCtNn8U_b7hhNTFgf3gJzXUEOtrn8L_uuthw7xG5LbY9Oro9myJvJKd2bZ84weNcA4EgTLVJ45AFVin1GecDXOkbJgXKf6pvqpQbBy_Fe8gxbM_A3ZJT6JrXKURhLempoczh7uTTM25y9ybmBJqnm7YjLTGRMy7DwoNIY6oZnVHWY_i04y5MPCDcGOVhn9bALuQ6s1XKZ6AsjEKbT3ytRKd2Q0LZnr7RSkfbk4LNYPtjKgQsVclC8Gao4t6-R76JebQn3GH65RNJr3zxogrDJk4OglgcaeasAou45tdfpBEj4l30DyYWUXcYqxNfsQxDlPaLGqEg30XR9bwKc8Kr57RFZ_16v5SccL6vYQoPO2g&refId=eGb0QUUekOjgqRmAiPqWmQ%3D%3D&trackingId=WfffkrTIdiq6gzEV2lzO5w%3D%3D&trk=flagship3_search_srp_jobs",
)

url_job_postings = JobMarketAssistant.scrap_job_postings_from_linkedin(job_posting_urls)
url_job_postings

,job_title,company,job_desc,location
0,Machine Learning Engineer,Ampstek,"Hi Professionals,Greetings From Ampstek!!Hope ...",Poland
1,Machine Learning Consultant,Antdata,Antdata is a data analytics company that focus...,"Poznań, Wielkopolskie, Poland"
2,Senior Data Scientist with Python and ML,Luxoft Poland,Project descriptionSuccessful candidate will b...,Poland
3,Senior Artificial Intelligence Engineer,Tangent,"About TangentAt Tangent, we partner with large...","Poznań, Wielkopolskie, Poland"
4,Machine Learning Researcher,RTB House,RTB House owes its market position to an advan...,"Warsaw, Mazowieckie, Poland"
5,AI Engineer,Opera,"Location: Wrocław, PolandAbout the job We are ...","Wrocław, Dolnośląskie, Poland"
6,AI Developer with Python,Rockwell Automation,Rockwell Automation is a global technology lea...,"Katowice, Śląskie, Poland"
7,Machine Learning Engineer,Devoteam G Cloud,"Company DescriptionDevoteam, a group of 8000+ ...","Warsaw, Mazowieckie, Poland"
8,Generative AI Engineer,Seargin,As a dynamic multinational tech company operat...,Poland
9,AIMl GenAI Manager,EY,AIML GenAI ManagerLet us introduce you the job...,"Wrocław, Dolnośląskie, Poland"


CVs have to be provided as plain text.


In [4]:
cv_1 = None
with open("data/cv_1.txt", "r") as file:
    cv_1 = file.read()

cv_2 = None
with open("data/cv_3.txt", "r") as file:
    cv_2 = file.read()

cv_3 = None
with open("data/cv_3.txt", "r") as file:
    cv_3 = file.read()

## Available methods


In [5]:
cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_1)

### Cover letter for Applied Data Scientist:
[Your Name]
[Address]
[City, State ZIP Code]
[Email Address]
[Today’s Date]

[Employer’s Name]
[Company Name]
[Address]
[City, State ZIP Code]

Dear [Employer’s Name],

I am writing to express my interest in the position of Applied Data Scientist at dunnhumby. As a highly motivated and skilled individual with a strong educational background and relevant work experience, I am confident that I have the necessary qualifications to contribute to your organization’s success.

My educational background includes a Bachelor’s degree in Artificial Intelligence, with a focus on courses like Data Visualization, NLP, ML, DL, BigData, Information Retrieval, Computer Vision, and Data Mining. I also have technical expertise in various programming languages, including Python, C++, SQL, R, and Java, and I am proficient in MS Office and have a valid driver’s license. Additionally, my fluency in English at a B2/C1 level allows me to effectively communicate and collaborate with colleagues and clients from diverse backgrounds.

I have held various positions that have allowed me to develop my skills and gain valuable experience, including my current role as a C++ Developer at CAD Projekt. I have also worked as a Cook at Prezzo and a Factory Worker at MSA Manufacturing. These experiences have honed my analytical thinking, problem-solving, and communication skills, as well as my ability to work effectively in a team environment.

I am particularly drawn to dunnhumby’s mission of empowering businesses to become advocates and champions for their Customers. Your innovative use of customer data to drive business growth and your commitment to diversity and inclusion resonate strongly with me. I am also eager to join your advanced data science team and utilize my skills in machine learning and human creativity to deliver impactful solutions for clients.

I am impressed by the comprehensive rewards package and the opportunity to work with cutting-edge technology that dunnhumby provides its employees. Your commitment to fostering a work-life balance for your employees through flexible working options further highlights your company’s inclusive culture.

As someone who values diversity and inclusivity, I appreciate your proactive approach to accommodating the needs of all candidates throughout the recruitment process. I believe that my experience and skills align perfectly with the qualifications you are seeking, and I am confident that I will make a valuable contribution to your team.

Thank you for considering my application. I look forward to the opportunity to contribute my skills and experience to dunnhumby and help drive the company’s mission forward. Please do not hesitate to contact me

In [10]:
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1)

/home/vrepetskyi/classes/6/nlp/job-market-assistant/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Jobs best matching the dream job "AI/ML C++ Developer", which was derived from the CV

,job_title,company,job_desc,location,score
0,AI Software Engineer (C++/GPGPU),microTECH Global LTD,The ideal candidate will have experience and e...,"Egham, England, United Kingdom",0.765772
1,C++ Developer (f/m/d) in Berlin and REMOTE 130...,think-cell Software,Algorithms\n\nthink-cell was founded on the id...,"Dublin, County Dublin, Ireland",0.762482
2,AI Software Engineer (C++/GPGPU),Extended Family International,6 Month Contract\n\nThe ideal candidate will h...,"Egham, England, United Kingdom",0.762315
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.753315
4,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.752523
5,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.736119
6,C++ Developer (f/m/d) in Berlin and REMOTE 130...,think-cell Software,Join us as a C++ Developer\n\n\n\n\nDo you bel...,"Dublin, County Dublin, Ireland",0.732527
7,Software Engineer (C++),Deep Render,Deep Render is a Deep Tech startup founded to ...,"London, England, United Kingdom",0.720320


### Missing skills:
1. Project management
2. Agile methodologies
3. Cloud computing
4. Serverless architecture
5. Natural language processing (NLP)
6. Database management
7. Data cleaning
8. Deep learning
9. Data visualization
10. Web development

## Benchmarks

TODO

### Experimenting with prompts

#### ChatGPT

In [ ]:
job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    dream_job_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
    postprocess_prompt="Deduplicate the following list of skills and limit it to 10 items: {missing_skills}",
)


cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_1)

missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1)

In [ ]:
job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    dream_job_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
    postprocess_prompt="Deduplicate the following list of skills and limit it to 10 items: {missing_skills}",
)


cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_1)

missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1)

In [ ]:
job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    dream_job_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
    postprocess_prompt="Deduplicate the following list of skills and limit it to 10 items: {missing_skills}",
)


cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_1)

missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1)

#### Gemini

#### Llama

### Model stability and execution time

#### ChatGPT

In [10]:
for _ in range(3):
    start_time = time.time()

    job_market_assistant = JobMarketAssistant(
        model=chat_gpt,
        cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
        dream_job_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
        key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
        missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
        postprocess_prompt="Deduplicate the following list of skills and limit it to 10 items: {missing_skills}",
    )

    cover_letter = job_market_assistant.get_cover_letter(
        file_job_postings.iloc[0], cv_1
    )

    missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1)

    display(Markdown(f"### Execution time: {time.time() - start_time} seconds"))

### Cover letter for Applied Data Scientist:
Dear Hiring Manager,

I am writing to express my interest in the Applied Data Scientist position at dunnhumby. With a strong background in both finance and accounting, as well as artificial intelligence, I am confident that I would be a valuable addition to your team.

After completing my degree in Automatics and Robotics in 2020, I decided to focus my studies on AI and completed my degree in Artificial Intelligence in 2025, with a focus on data visualization, NLP, ML, DL, BigData, Information Retrieval, Computer Vision, and Data Mining. I have also gained practical experience through my employment as a C++ Developer at CAD Projekt in Poznań. My skills in programming languages such as Python, C++, SQL, R, and Java, as well as my proficiency in MS Office and my driving license, make me well-equipped for this role.

During my academic career, I have also had the opportunity to study abroad through an Erasmus program, which has enhanced my cultural awareness and ability to work with diverse teams. I am also a proud member of GHOST, an academic circle focused on AI/ML, where I have gained valuable leadership skills as a section organizer for GHOST’s conference on ML.

As a data-driven company, dunnhumby’s mission to empower businesses to become advocates for their Customers resonates strongly with me. I am excited about the opportunity to apply my expertise in distilling complex problems into insightful solutions using machine learning and human creativity. Working alongside other highly skilled data scientists and challenging the status quo sounds like a rewarding and fulfilling experience.

In addition to my skills and experience, I also possess strong communication skills, logical thinking, and problem-solving abilities, as well as a passion for connecting data science with the customer experience. I am confident that I would be a valuable asset in developing and deploying complex applications and components that drive meaningful impact for clients.

I am impressed by dunnhumby's commitment to diversity and inclusion, and I believe in the importance of work-life balance. As such, I appreciate your dedication to providing flexible working options and creating an inclusive culture.

Thank you for considering my application. I am eager to learn more about the role and how I can contribute to dunnhumby's mission. Please do not hesitate to contact me to discuss further.

Sincerely,
[Your Name]

/home/vrepetskyi/classes/6/nlp/job-market-assistant/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI-focused C++ Developer, which was derived from the CV

,job_title,company,job_desc,location,score
0,C++ Developer (f/m/d) in Berlin and REMOTE 130...,think-cell Software,Algorithms\n\nthink-cell was founded on the id...,"Dublin, County Dublin, Ireland",0.775290
1,AI Software Engineer (C++/GPGPU),microTECH Global LTD,The ideal candidate will have experience and e...,"Egham, England, United Kingdom",0.764987
2,AI Software Engineer (C++/GPGPU),Extended Family International,6 Month Contract\n\nThe ideal candidate will h...,"Egham, England, United Kingdom",0.756969
3,C++ Developer (f/m/d) in Berlin and REMOTE 130...,think-cell Software,Join us as a C++ Developer\n\n\n\n\nDo you bel...,"Dublin, County Dublin, Ireland",0.738293
4,Software Engineer (C++),Deep Render,Deep Render is a Deep Tech startup founded to ...,"London, England, United Kingdom",0.734283
5,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.699809
6,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.694832
7,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.694573


### Missing skills:
5. **Digital Marketing Specialist**
- Search Engine Optimization (SEO)
- Social Media Marketing
- Content Creation/Copywriting
- Email Marketing
- Pay-Per-Click (PPC) Advertising
- Analytics and Reporting
- Conversion Rate Optimization
- Marketing Automation

The 10 skills are:

1. Data Scientist
2. Machine Learning
3. Software Engineering
4. Financial Analysis
5. Robotics Engineering
6. Data Visualization
7. Web Development
8. Accounting
9. Digital Marketing
10. Statistical Analysis

### Execution time: 41.497992515563965 seconds

#### Gemini

#### Llama